# Logging

## Set up

In [3]:
from requests import get

The following cell runs a Docker container that utilizes a specified file to build a FastAPI application.

In [47]:
!docker run --rm -itd\
    --name test_container\
    -v ./logging_files/app.py:/app.py\
    -p 8000:8000 \
    fastapi_experiment \
    uvicorn --host 0.0.0.0 --reload app:app

f68905f3f51c17708254088d45188c19ed78f1839c2e6c4e4751f4d8f24f068b


Don't forget to stop the container.

In [46]:
!docker stop test_container

test_container


## View logs

Let's see what the default logs of the fast api look like. We will make an application to write something to the console and then extract it from the docker container.

In the following cell we've just defined a simple fastapi program that throws a `HTTPException` if called with a `False` argument.

In [1]:
%%writefile logging_files/app.py
from fastapi import FastAPI, HTTPException

app = FastAPI()

@app.get("/{is_ok}")
def say_hello(is_ok : bool):
    if is_ok:
        return "im fine"
    else:
        raise HTTPException(status_code=404, detail="Item not found")

Overwriting logging_files/app.py


Now let's send some requests to the application and view it's log last lines:

In [4]:
get("http://localhost:8000/True")
get("http://localhost:8000/False")
!docker logs --tail 2 test_container

INFO:     172.17.0.1:39106 - "GET /True HTTP/1.1" 200 OK
INFO:     172.17.0.1:39116 - "GET /False HTTP/1.1" 404 Not Found
